# Preprocessing
- Stem the Data (run/running/ran --> ran)
- Identify the most commonly used words in decription
- One hot encody top top 10(ish) of the nontrvial words
- Scale the Data
- PCA (95% of variation captured)

# Models

- Linear Regression
- Decision Tree Regression
- [Plot Tree Regression](https://scikit-learn.org/stable/auto_examples/tree/plot_tree_regression.html)
- Perceptron
- Neural Network

# Set Hyperparameters

Use 10-fold cross validation to tune hyperparameters
- using RSS as an error metric?

In [1]:
import pandas as pd
import numpy as np
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords#stopwords are words like "is", and "the"
from sklearn.model_selection import train_test_split
from tqdm import tqdm #pip install tqdm, loading bar for visualization
import string#to access a convinient list of punctiation
import re#to replace numbers in strings
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer

In [2]:

df = pd.read_csv("data/winemag-data-130k-v2.csv", index_col=0)

In [3]:
df

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
...,...,...,...,...,...,...,...,...,...,...,...,...,...
129966,Germany,Notes of honeysuckle and cantaloupe sweeten th...,Brauneberger Juffer-Sonnenuhr Spätlese,90,28.0,Mosel,NaN,NaN,Anna Lee C. Iijima,NaN,Dr. H. Thanisch (Erben Müller-Burggraef) 2013 ...,Riesling,Dr. H. Thanisch (Erben Müller-Burggraef)
129967,US,Citation is given as much as a decade of bottl...,NaN,90,75.0,Oregon,Oregon,Oregon Other,Paul Gregutt,@paulgwine,Citation 2004 Pinot Noir (Oregon),Pinot Noir,Citation
129968,France,Well-drained gravel soil gives this wine its c...,Kritt,90,30.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,Gewürztraminer,Domaine Gresser
129969,France,"A dry style of Pinot Gris, this is crisp with ...",NaN,90,32.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Marcel Deiss 2012 Pinot Gris (Alsace),Pinot Gris,Domaine Marcel Deiss


In [4]:
for attr in df:
    print(f"{attr}: {round((sum(df[attr].isnull())/df.shape[0])*100,4)}%")

country: 0.0485%
description: 0.0%
designation: 28.8257%
points: 0.0%
price: 6.9215%
province: 0.0485%
region_1: 16.3475%
region_2: 61.1367%
taster_name: 20.1922%
taster_twitter_handle: 24.0154%
title: 0.0%
variety: 0.0008%
winery: 0.0%


In [25]:
def text_stemming(df):
    stop_words = set(stopwords.words("english"))
    unhelpful_words = set(["wine", "drink"])

    stemmer = PorterStemmer()
    xFeat = df.to_numpy()
    newDescription = []
    l = len(xFeat)
    for i in tqdm(range (l), desc="Stemming"):
        wordList = word_tokenize(re.sub(r'\d+', 'number', (xFeat[i].lower()).translate(str.maketrans('', '', string.punctuation)))) #splits the words into a list, and lowercases them
        description = []
        for item in wordList:
            if item not in stop_words:#we can get rid of stop words: "is", "a", "the"
                candidate = stemmer.stem(item)
                if candidate not in unhelpful_words:
                    description.append(candidate) #replace each word with its stem
        newDescription.append(np.array(candidate))
    return pd.DataFrame(newDescription)

In [26]:
df["description"] = text_stemming(df["description"])

Stemming:  93%|█████████▎| 112576/120975 [00:08<00:00, 12614.45it/s]


AttributeError: 'float' object has no attribute 'lower'

In [20]:
df.dropna(subset=['price'], inplace=True)

In [23]:
df

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
1,Portugal,ferment,Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,finish,NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,stew,Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,grabbi,Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
5,Spain,tannin,Ars In Vitro,87,15.0,Northern Spain,Navarra,NaN,Michael Schachner,@wineschach,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem
...,...,...,...,...,...,...,...,...,...,...,...,...,...
129966,Germany,NaN,Brauneberger Juffer-Sonnenuhr Spätlese,90,28.0,Mosel,NaN,NaN,Anna Lee C. Iijima,NaN,Dr. H. Thanisch (Erben Müller-Burggraef) 2013 ...,Riesling,Dr. H. Thanisch (Erben Müller-Burggraef)
129967,US,NaN,NaN,90,75.0,Oregon,Oregon,Oregon Other,Paul Gregutt,@paulgwine,Citation 2004 Pinot Noir (Oregon),Pinot Noir,Citation
129968,France,NaN,Kritt,90,30.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,Gewürztraminer,Domaine Gresser
129969,France,NaN,NaN,90,32.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Marcel Deiss 2012 Pinot Gris (Alsace),Pinot Gris,Domaine Marcel Deiss


# Vectorize

In [21]:
df.to_csv("data/data_stemmed.csv", index = False)

In [22]:
stemmed = pd.read_csv("data/data_stemmed.csv")
stemmed.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Portugal,ferment,Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
1,US,finish,NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
2,US,stew,Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
3,US,grabbi,Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
4,Spain,tannin,Ars In Vitro,87,15.0,Northern Spain,Navarra,NaN,Michael Schachner,@wineschach,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem


In [11]:
vectorizer = CountVectorizer(analyzer='word', tokenizer=word_tokenize, ngram_range=(1, 1), min_df=1)
print("Vectorizing...")
splitWords = vectorizer.fit_transform(stemmed['description'])
vocabMap = list(vectorizer.get_feature_names())
# Taken from http://stackoverflow.com/questions/3337301/numpy-matrix-to-array
# and http://stackoverflow.com/questions/13567345/how-to-calculate-the-sum-of-all-columns-of-a-2d-numpy-array-efficiently
counts = splitWords.sum(axis=0).A1
finalMap = Counter(dict(zip(vocabMap, counts)))
print(finalMap.most_common(10))

Vectorizing...
[('flavor', 66947), ('fruit', 53469), ('number', 42263), ('finish', 38962), ('aroma', 38838), ('palat', 36574), ('acid', 35694), ('cherri', 30136), ('tannin', 29951), ('ripe', 25838)]


In [12]:
print(finalMap.most_common(31))

[('flavor', 66947), ('fruit', 53469), ('number', 42263), ('finish', 38962), ('aroma', 38838), ('palat', 36574), ('acid', 35694), ('cherri', 30136), ('tannin', 29951), ('ripe', 25838), ('note', 23849), ('black', 23809), ('dri', 22821), ('spice', 21245), ('rich', 19644), ('fresh', 19229), ('red', 17784), ('show', 17208), ('oak', 16760), ('berri', 16548), ('nose', 16322), ('offer', 14990), ('plum', 14736), ('blend', 14347), ('blackberri', 14337), ('textur', 14056), ('sweet', 13827), ('appl', 13747), ('balanc', 13451), ('soft', 12904), ('crisp', 12422)]


In [13]:
 xTrain, xTest, yTrain, yTest = train_test_split(df.loc[:, df.columns != 'price'], df['price'], test_size =.3, random_state = 334)

# Neural Network

Using RNN

In [14]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, Dataset, DataLoader, random_split
from torch.nn import functional as F

if not torch.cuda.is_available():
    raise EnvironmentError("CUDA not available, skipping neural network")

In [17]:
xTrain['description'].ravel()

k spice firm palat evok blackberri jam tobacco green pepper alongsid smooth tannin enjoy number ',
 'ripe stout smoki darkberri aroma spici bit hot lush merlot warm year volumin chubbi heft full extract spici toasti blackberri cassi flavor come chocolati oak dens chewi mouth coat finish number ',
 'full bitter marmalad mapl syrup flavor also show consider woodag flavor structur still show hint fruit also bring strongli matur charact ',
 'aroma lee citru almond melon appl spice butter draw glass bring dazzl sens textur fruit spice flavor dappl throughout ',
 'garnet eye zippi nose cherri cranberri flavor pomegran cranberri tart cherri nestl wellstructur tannin seem dissolv palat bright finish take hold ',
 'green manur note gener barnyard aroma provid aromat interest ’ make excess tart cherri fruit oddli cloy finish ',
 'lightli fruiti hint strawberri fruit lace soft acid bitter background pleasant ',
 'dens tannic dri bitter dark cocoa black fruit flavor concentr fruit tannin impress g